# AIML231 Assignement One - Part Three
> Implementation of the Decision Tree classifier

# Imports

In [ ]:
import pandas as pd
import numpy as np
from math import log2

## Node Class Definition 

In [ ]:
class DecisionTreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, is_leaf=False, label=None, depth=0):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.label = label
        self.depth = depth

    
    def string_representation(self):
        if self.is_leaf:
            return ''.join(self.depth*['  '],) + f'leaf:{self.feature}={self.value}&label={self.label}&depth={self.depth}'
        else:
            return ''.join(self.depth*['  '],) + f'{self.feature}={self.value}&depth={self.depth}\n->left  ' + self.left.string_representation() + '\n->right ' + self.right.string_representation()

## Classifier Definition

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=3, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    
    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    
    def _build_tree(self, X, y, depth):
        # Stop splitting if max depth is reached, if all targets are the same, or if the dataset size is below min_samples_split
        if depth == self.max_depth or len(set(y)) == 1 or len(y) < self.min_samples_split: 
            return DecisionTreeNode(is_leaf=True, label=max(set(y), key=list(y).count), depth=depth)
    
        best_feature, best_value = self._find_best_split(X, y)
        left_idx = X[best_feature] == best_value
        right_idx = ~left_idx
        
        # Further checks to ensure each child node adheres to min_samples_split
        if np.sum(left_idx) < self.min_samples_split or np.sum(right_idx) < self.min_samples_split:
            return DecisionTreeNode(is_leaf=True, label=max(set(y), key=list(y).count), depth=depth)
    
        left_child = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_child = self._build_tree(X[right_idx], y[right_idx], depth + 1)
    
        return DecisionTreeNode(feature=best_feature, value=best_value, left=left_child, right=right_child, depth=depth)
    
        
    def _find_best_split(self, X, y):
        best_feature, best_value, best_gain = None, None, 0
        for feature in X.columns:
            values = list(set(X[feature]))
            values.sort()
            for value in values:
                left_idx = X[feature] == value
                right_idx = ~left_idx
                gain = self._information_gain(y, y[left_idx], y[right_idx])
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_value = value
        return best_feature, best_value

    
    def _information_gain(self, parent, left, right):
        """Calculate the information gain of a split."""
        
        def entropy(y):
            """Calculate the entropy of a label distribution."""
            # Count occurrences of each label
            counts = {}
            for label in y:
                if label in counts:
                    counts[label] += 1
                else:
                    counts[label] = 1
            
            # Calculate probabilities
            probabilities = [count / len(y) for count in counts.values()]
            
            # Calculate entropy
            return -sum(p * log2(p) for p in probabilities if p > 0)
            
        parent_entropy = entropy(parent)
        left_entropy = entropy(left)
        right_entropy = entropy(right)
        
        # Calculate the weighted average child entropy
        child_entropy = (len(left) / len(parent)) * left_entropy + (len(right) / len(parent)) * right_entropy
        
        # Information gain is the reduction in entropy
        return parent_entropy - child_entropy

    
    def predict(self, X):
        tmp_pred = []
        for index, row in X.iterrows():
            tmp_pred.append(self._predict_sample(self.root, row))
        return tmp_pred

    
    def _predict_sample(self, node, sample):
        """Recursively predict the class of a sample based on the decision tree."""
        if node.is_leaf:
            return node.label
    
        # Traverse the tree based on the split condition
        if sample[node.feature] == node.value:
            return self._predict_sample(node.left, sample)
        else:
            return self._predict_sample(node.right, sample)

## Training

In [ ]:
### Code to test the learning of decision trees.
data = pd.DataFrame({
    'Outlook': ['s', 's', 'o', 'r', 'r', 'r', 'o', 's', 's', 'r', 's', 'o', 'o', 'r'],
    'Humidity': ['h', 'h', 'h', 'h', 'n', 'n', 'n', 'h', 'n', 'n', 'n', 'h', 'n', 'h'],
    'Wind': ['w', 's', 'w', 'w', 'w', 's', 's', 'w', 'w', 'w', 's', 's', 'w', 's'],
    'PlayTennis': [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]
})

# Train the decision tree classifier
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(data[['Outlook', 'Humidity', 'Wind']], data['PlayTennis'])

## Predictions

In [ ]:
# Make predictions
predictions = clf.predict(
    pd.DataFrame({
        'Outlook': ['o', 's', 'r', 'o'],
        'Humidity': ['n', 'h', 'h', 'n'],
        'Wind': ['w', 's', 's', 'w']
        }
    )
)

predictions

## Tree Representation 

In [ ]:
print(clf.root.string_representation())

## Graphical Tree Representation 

In [ ]:
from util.plot_tree import plot_decision_tree

In [ ]:
plot_decision_tree(clf.root)